In [7]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.cluster import KMeans
import numpy as np
import pycountry

In [8]:
#!/usr/bin/env /Applications/MAMP/Library/bin/python

import mysql.connector

config = {
  'user': 'root',
  'password': 'root',
  'host': '127.0.0.1',
  'port': 8889,
  'database': 'ecotourisme',
  'raise_on_warnings': True
}

cnx = mysql.connector.connect(**config)

cursor = cnx.cursor(dictionary=True)

cursor.execute('SELECT pays.nom as pays, s.id_pays, s.annee, s.gpi, t.arriveesTotal as arriveesTotal, e.pibParHab, eco.co2, eco.GES_hab as gesParHab, dv.Value FROM surete as s, tourisme as t, economie as e, ecologie as eco, pays, developpement_humain as dv where pays.id = s.id_pays and s.id_pays = t.id_pays AND t.id_pays = e.id_pays AND e.id_pays = eco.id_pays AND eco.id_pays = dv.iso_code AND s.annee = t.annee AND t.annee = e.annee AND e.annee = eco.annee AND e.annee=2019;')
results = cursor.fetchall()


data = []
for row in results:
    pays = row['pays']
    id_pays = row['id_pays']
    annee = row['annee']
    gpi = row['gpi']
    arriveesTotal = row['arriveesTotal']
    pibParHab = row['pibParHab']
    co2 = row['co2']
    gesParHab = row['gesParHab']
    idh = row['Value']
    data.append({'pays': pays,'id_pays': id_pays, 'arriveesTotal': arriveesTotal, 'gpi': gpi, 'pibParHab': pibParHab, 'co2': co2, 'gesParHab': gesParHab, 'idh': idh, 'annee': annee})


cnx.close()

In [9]:
# Créer le DataFrame
df = pd.DataFrame(data)

# Normaliser les indicateurs par la méthodes min/max
df['arriveesTotal'] = df['arriveesTotal'] / df['arriveesTotal'].max()
df['gpi'] = df['gpi'].min() / df['gpi']
df['pibParHab'] = df['pibParHab'] / df['pibParHab'].max()
df['co2'] = df['co2'].min() / df['co2']
df['gesParHab'] = df['gesParHab'].min() / df['gesParHab']
df['idh'] = df['idh'].astype(str).str.replace(',', '.').astype(float)
df['idh'] = df['idh'] / df['idh'].max()

# Supprimer les valeurs manquantes
df = df.dropna()

CLUSTERING 

In [10]:
# Sélectionner les variables pour le clustering
X = df[['arriveesTotal', 'gpi', 'pibParHab', 'co2', 'gesParHab', 'idh']]

# Créer le modèle
kmeans = KMeans(n_clusters=8, random_state=42)

# Faire le clustering
kmeans.fit(X)

# Ajouter les labels de clusters dans le DataFrame
df['cluster'] = kmeans.labels_

df = df.sort_values(by='cluster', ascending=False)

centroids = pd.DataFrame(kmeans.cluster_centers_, columns=X.columns)

# Afficher les centroïdes
centroids

# Afficher le DataFrame
df.head()

,pays,id_pays,arriveesTotal,gpi,pibParHab,co2,gesParHab,idh,annee,cluster
131,Sierra Leone,SL,0.000326,0.601541,0.016849,0.226608,0.556339,0.495842,2019,7
122,Malawi,MW,0.004489,0.600880,0.015445,0.143213,0.619243,0.532225,2019,7
118,Rwanda,RW,0.007500,0.560513,0.022293,0.162222,0.885834,0.555094,2019,7
116,Haïti,HT,0.004305,0.533952,0.031562,0.070960,0.515709,0.556133,2019,7
36,Slovaquie,SK,0.073831,0.719552,0.325316,0.007586,0.107253,0.881497,2019,6


In [11]:
# Fonction pour convertir ISO-2 en ISO-3
def iso2_to_iso3(iso2):
    try:
        return pycountry.countries.get(alpha_2=iso2).alpha_3
    except (AttributeError, LookupError):
        return np.nan
    
# Appliquer la fonction à la colonne 'id_pays'
df['id_pays'] = df['id_pays'].apply(iso2_to_iso3)

# Créer une carte du monde
fig = px.choropleth(df, locations='id_pays', color='cluster', hover_name='pays', projection='natural earth')

# Mettre à jour la carte
fig.update_geos(showcountries=True, countrycolor="Black", showland=True, showocean=True, oceancolor="LightBlue", showlakes=True, lakecolor="Blue")

# Afficher la carte
fig.show()


In [12]:
# Créer une figure
fig = go.Figure()

# Ajouter les points pour chaque cluster
for cluster in df['cluster'].unique():
    cluster_df = df[df['cluster'] == cluster]
    fig.add_trace(go.Scatter3d(x=cluster_df['pibParHab'],
                               y=cluster_df['co2'],
                               z=cluster_df['gesParHab'],
                               mode='markers+text',
                               marker=dict(size=4),
                               name=f'Cluster {cluster}'))
    
# Ajouter les centroïdes
fig.add_trace(go.Scatter3d(x=centroids['pibParHab'],
                           y=centroids['gesParHab'],
                           z=centroids['gpi'],
                           mode='markers',
                           marker=dict(size=6, color='black', symbol='x'),
                           name='Centroid'))

# Mettre à jour le nom des axes 
fig.update_scenes(xaxis_title='pibParHab',
                  yaxis_title='gesParHab',
                  zaxis_title='GPI')

# Afficher la figure
fig.show()